## 15.1 스파크 애플리케이션의 아키텍쳐

- 스파크 드라이버
  - 스파크 애플리케이션의 '운전자 역할'을 하는 프로세스.
  - 드라이버는 스파크 애플리케이션의 실행을 제어하고 스파크 클러스터의 모든 상태 정보 유지.
  - 물리적 컴퓨팅 자원 확보와 익스큐터 실행을 위해 클러스터 매니저와 통신할 수 있어야 함.
- 스파크 익스큐터
  - 스파크 드라이버가 할당한 태스크를 수행하는 프로세스
  - 드라이버가 할당한 태스크를 받아 실행하고 태스크의 상태와 결과(성공, 실패)를 드라이버에 보고
  - 모든 스파크 애플리케이션은 개별 익스큐터 프로세스 사용
- 클러스터 매니저
  - 스파크 애플리케이션을 실행할 클러스터 머신을 유지함. 
  - '드라이버'(또는 마스터)와 '워커'라는 개념을 가지고 있음
  - 클러스터 매니저는 물리적인 머신에 연결되는 개념

- 스파크 애플리케이션은 실제로 실행할 때가 되면 클러스터 매니저에 자원 할당 요청을 함
- 사용자 애플리케이션 설정에 따라 스파크 드라이버를 실행할 자원을 포함해 요청하거나 스파크 애플리케이션 실행을 위한 익스큐터 요청 가능
- 스파크 애플리케이션 실행 과정에서 클러스터 매니저는 애플리케이션이 실행되는 머신 관리

- 스파크가 지원하는 클러스터 매니저
  - 스탠드얼론 클러스터 매니저
  - 아파치 매소스
  - 하둡 YARN
  
 <br>
 
### 15.1.1 실행 모드

- 선택 가능한 실행 모드
  - 클러스터 모드
  - 클라이언트 모드
  - 로컬 모드

- 클러스터 모드
  - 클러스터 모드를 사용하려면 컴파일 된 JAR 파일이나 파이썬 스크립트 또는 R 스크립트 클러스터 매니저에 전달해야함
  - 받은 파일은 다음 워커 노드에 드라이버와 익스큐터 프로세스를 실행함
  
- 클라이언트 모드
  - 애플리케이션을 제출한 클라이언트 머신에 스파크 드라이버가 위치한다는 것을 제외하면 클러스터 모드와 비슷함. 
  - 스파크 드라이버 프로세스를 유지하며 클러스터 매니저는 익스큐터 프로세스를 유지함
  - 게이트웨이 머신 또는 에지 노드라는 실행 머신이 있음
  - 외부의 머신 (게이트웨이 머신, 에지 노드)에서 실행되고 나머지 워커는 클러스터에 위치
  
- 로컬 모드
  - 모든 스파크 애플리케이션이 단일 머신에서 실행됨
  - 애플리케이션의 병렬 처리를 위해 단일 머신의 스레드를 사용함
  - 애플리케이션 테스트 또는 개발 중인 애플리케이션을 반복적으로 실험하는 용도로 사용됨
 

## 15.2 스파크 애플리케이션의 생애주기(스파크 외부)

<br>

### 15.2.1 클라이언트 요청
- 스파크 애플리케이션(컴파일된 JAR나 라이브러리 파일) 제출을 최우선적으로 해야함
- 제출하는 시점에 로컬머신에서 코드가 실행되어 클러스터 드라이버 노드에 요청함 -> 스파크 드라이버 프로세스의 자원을 함께 요청
- 클러스터 매니저는 요청을 받고 클러스터 노드 중 하나의 드라이버 프로세스를 실행함
- 스파크 잡을 제출한 클라이언트 프로세스는 종료되고 애플리케이션은 클러스터에서 실행됨

<br>

### 15.2.2 시작
- 사용자 코드를 실행시키기 위해서는 반드시 스파크 클러스터를 초기화하는 <code>SparkSession</code>를 포함해야함
- <code>SparkSession</code>은 클러스터 매니저와 통신해 스파크 익스큐터 프로세스의 실행을 요청함
- <code>spark-submit</code>을 실행할 때 사용하는 명령행 인수로 익스큐터 수와 설정값을 지정함
- 이를 통해 스파크 클러스터 생성

<br>

### 15.2.3. 실행
- 드라이버와 워커는 코드를 실행하고 데이터를 이동하는 과정에서 서로 통신.
- 드라이버는 각 워커에 태스크를 할당
- 태스크를 할당 받은 워커는 태스크의 상태와 성공/실패 여부를 드라이버에 전송

### 15.2.4 완료
- 스파크 애플리케이션 실행이 완료되면 드라이버 프로세스가 성공 또는 실패의 상태로 완료
- 클러스터 매니저는 드라이버가 속한 스파크 클러스터의 모든 익스큐터를 종료시킴

## 15.3 스파크 애플리케이션의 생애주기(스파크 내부)
- 스파크 애플리케이션은 하나 이상의 스파크 잡으로 구성
- 스레드를 사용해 여러 액션을 병렬로 수행하는 경우가 아니면 애플리케이션의 스파크 잡은 차례대로 실행

### 15.3.1 SparkSession
- 대화형 모드에서는 자동으로 생성되지만, 애플리케이션을 만드는 경우에는 직접 생성해야함
- <code>new SparkContext</code>보다는 <code>SparkSession</code>의 빌더 메서드를 추천
  - 스파크와 스파크 sql context를 안전하게 생성할 수 있음
  - context 충돌을 방지할 수 있음

In [ ]:
## ex
from pysark.sql import SparkSession

spark = SparkSession.builder.master('local').appName('word count').config('spark.some.config.option', 'some-value')\
.getOrCreate()

- <code>SparkSession</code>를 생성하면 스파크 코드를 실행할 수 있음.
- 이를 활용해서 저수준 API, 기존 context 그리고 관련 설정 정보에 접근할 수 있음
#### SparkContext
- <code>SparkSession</code>의 <code>SparkContext</code>는 스파크 클러스터에 대한 연결을 나타냄
- <code>SparkContext</code>로 RDD, 어큐뮬레이터 그리고 브로드캐스트 변수를 생성하고 코드를 실행할 수 있음
- 명시적으로 초기화 할 필요 없음, 직접 초기화 하는 방식은 <code>getOrCreate</code> 방식 사용

### 15.3.2 논리적 명령

#### 논리적 명령을 물리적 실행 계획으로 변환하기

In [ ]:
## ex
df1 = spark.range(2,1000000, 2)
df2 = spark.range(2,1000000, 4)
step1 = df1.repartition(5)
step12 = df1.repartition(6)
step2 = step1.selectExpr('id * 5 as id')
step3 = step2.join(step12, ['id'])
step4 = step3.selectExpr('sum(id)')

step4.collect()

- collect 같은 액션을 호출하게 되면 개별 스테이지와 태스크로 이루어진 스파크 잡이 실행.

### 15.3.3 스파크 잡
- 보통 액션 하나당 하나의 스파크 잡이 생성, 액션은 항상 결과를 반환
- 스파크 잡은 일련의 스테이지로 나뉘고 스테이지 수는 셔플 작업이 얼마나 많이 발생하는지에 따라 달라짐

### 15.3.4 스테이지
- 다수의 머신에서 동일한 연산을 수행하는 태스크의 그룹을 나타냄
- 가능한 한 많은 태스크를 동일한 스테이지로 묶으려고 함
- 셔플 작업이 일어난 후에는 반드시 새로운 스테이지 시작 (셔플은 데이터의 물리적 재분배 과정)
- 파티션을 재분배 하는 과정은 데이터를 이동시키는 작업이므로 익스큐터 간의 조정이 필요
- 셔플이 끝난 뒤에는 새로운 스테이지 시작함. 최종 결과를 계산하기 위해 스테이지 실행 순서 지속적으로 추적
- 클러스터 익스큐터 수보다 파티션 수를 더 크게 지정하는 것이 유리함. 

### 15.3.5 태스크
- 스파크의 스테이지는 태스크로 구성
- 각 태스크는 단일 익스큐터에서 실행할 데이터의 블록과 다수의 트랜스포메이션 조합
- 태스크는 데이터 단위(파티션)에 적용되는 연산 단위
- 파티션을 늘리면 병렬성을 높일 수 있다. 

## 15.4 세부 실행 과정
- 스파크는 <code>map</code> 연산 후 다른 <code>map</code> 연산이 이어진다면 함께 실행할 수 있도록 스테이지와 태스크를 자동으로 연결
- 모든 셔플을 작업할 때 데이터를 안정적인 저장소에 저장하므로 여러 잡에서 재사용할 수 있음


### 15.4.1 파이프라이닝
- 노드 간의 데이터 이동 없이 각 노드가 데이터를 직접 공급할 수 잇는 연산만 모아 태스크의 단일 스테이지로 만듦. 
- 파이프라인으로 구성된 연산 작업은 단계별로 메모리나 디스크에 중간 결과를 기록하는 방식보다 훨씬 더 처리 속도가 빠름
### 15.4.2 셔플 결과 저장
- 노드 간 복제를 유발하는 연산을 실행하면 엔진에서 파이프라이닝을 수행하지 못하므로 네트워크 셔플 발생
- 노드 간 복제를 유발하는 연산은 각 키에 대한 입력 데이터를 먼저 여러 노드로부터 복사함
- 셔플 파일을 로컬 디스크에 기록
- 잡이 실패할 경우 셔플 파일을 저장했기 때문에 '소스'스테이지가 아닌 해당 스테이지부터 처리 가능
- 실패한 리듀스 태스크부터 재실행 가능
- 다만 셔플 파일에 담겨 있는 이전 스테이지는 생략하고 진행됨